# 第七章 规划和股票报告生成
在这章我们将会搭建一个定制的多Agent群聊，并让他们合作生成关于过去一个月股票表现的详细报告。
<center>
<img src="images/group_of_agents.jpg" width="600">
</center>

## 一、设置配置

In [1]:
llm_config={"model": "gpt-4-turbo"}

## 二、创建任务

In [2]:
task = "写一篇关于过去一个月"\
"Nvidia股价表现的博客。今天是 2024-04-23。"

## 三、建立群聊

在之前的学习中，我们学习了顺序聊天的对话模版，它可以执行多步任务，但需要人类设计具体的步骤和每个步骤中涉及的具体Agent，如果我们想要简单地定义Agent并让它们共同解决这个任务，那么这个群聊的对话模式能够满足你的需求。

为了完成任务，这个群聊将包含以下Agent：

1. **User_proxy** or **Admin**: 允许用户对报告进行评论，并要求作者对报告进行改进。
2. **Planner**: 确定完成任务所需的相关信息。
3. **Engineer**: 使用Planner定义的计划编写代码。
4. **Executor**: 执行Engineer编写的代码。
5. **Writer**: 撰写报告
接下来我们逐个定义上述Agengt。

In [3]:
import autogen

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In [4]:
user_proxy = autogen.ConversableAgent(
    name="Admin",                       # 管理员
    system_message="给出任务，并发送指令给writer，从而完善博客文章。",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",          # 将人类输入模式设置为 ALWAYS 每次轮到Admin Agent说话时，始终要求人类输入
)                                       # 如果跳过输入，Admin Agent会发送指示给Writer Agengt来改进博文

In [5]:
planner = autogen.ConversableAgent(
    name="Planner",              # 规划者
    system_message="给定一项任务，请用中文确定完成该任务需要哪些信息。"
    "请注意，所有信息都将使用Python代码检索。"
    "在其他人完成每一步后，检查进度并指示剩余步骤。"
    "如果某一步失败，尝试解决。",
    description="Planner. 给定一项任务，确定完成该任务需要哪些信息。"
    "在其他人完成每一步后，检查进度并指示剩余步骤 ",
    llm_config=llm_config,
)

In [6]:
engineer = autogen.AssistantAgent(
    name="Engineer",            # 工程师                   
    llm_config=llm_config,
    description="基于planner的计划写代码。"
)

In [7]:
executor = autogen.ConversableAgent(
    name="Executor",             # 执行者
    system_message="执行engineer的代码并且返回结果。",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,   # 能看到的执行代码结果的消息数
        "work_dir": "coding",   # 代码保存路径
        "use_docker": False,    # 是否使用docker执行代码
    },
)

In [8]:
writer = autogen.ConversableAgent(
    name="Writer",               # 作家/写作者
    llm_config=llm_config,
    system_message="Writer."
    "请用markdown格式写博客(带有相关标题)，并将内容放在伪``` `md```代码块中。"
    "你从管理员那里得到反馈并改进你的博客。",
    description="Writer."          # 作家/写作者
    "根据代码执行结果编写博客，并根据管理员的反馈完善博客。"
)

## 四、定义群聊
将之前定义好的Agent拉进群聊。

In [9]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],   # agents为参与群聊的Agent，接收格式为List[Agent]
    messages=[],   
    max_round=10,   
)

定义群聊管理器，我们要通过Admin对群聊管理器提出任务从而开始对话，同样的群聊管理器也需要配置 `llm_config`。

In [10]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

## 五、开始群聊

In [11]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

写一篇关于过去一个月Nvidia股价表现的博客。今天是 2024-04-23。

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

为了写一篇关于过去一个月Nvidia股价表现的博客，我们需要以下信息：

1. **获取Nvidia的股票代码** - 确认Nvidia的股票代码（通常是NVDA）。
2. **获取历史股票数据** - 使用Python代码获取过去一个月的Nvidia股价数据（具体日期范围：2024-03-23至2024-04-23）。
3. **数据处理和可视化** - 对数据进行处理，并创建图表以可视化股价趋势。
4. **市场和行业背景信息** - 获取Nvidia在过去一个月的新闻和行业背景信息，以了解可能影响股价变动的原因。
5. **撰写博客内容** - 将分析结果、数据图表和市场背景信息整合成文，将其撰写成一篇流畅的博客文章。

接下来，我们将按照步骤进行：

### 第一步：确认Nvidia的股票代码
```python
# Nvidia 的股票代码查询
nvidia_ticker = 'NVDA'
print(nvidia_ticker)
```

接下来，请运行上述代码以确认Nvidia的股票代码。如果确认无误，请继续进行第二步。

--------------------------------------------------------------------------------

Next speaker: Engineer

Engineer (to chat_manager):

接下来，我们获取过去一个月的Nvidia股价数据并进行可视化处理。我们将使用Python编写代码，通过Python库如 `yfinance` 获取历史股价数据。

### 第二步：获取历史股票数据

```python
# filename: fetch_nvidia_stock_data.py

import yf

## 六、添加说话人选择策略

In [12]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="给出任务，并发送指令给作者，从而完善博客文章。",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

planner = autogen.ConversableAgent(
    name="Planner",
    system_message="给定一项任务，请用中文确定完成该任务需要哪些信息。"
    "请注意，所有信息都将使用Python代码检索。"
    "在其他人完成每一步后，检查进度并指示剩余步骤。"
    "如果某一步失败，尝试解决。",
    description="给定一项任务，确定完成该任务需要哪些信息。"  # 
    "在其他人完成每一步后，检查进度并指示剩余步骤",
    llm_config=llm_config,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="基于planner的计划写代码。",
)

writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer. "
    "请用markdown格式写博客(带有相关标题)，并将内容放在伪``` `md```代码块中。"      
    "从admin那里得到反馈并改进你的博客。",
    description="根据代码执行结果编写博客，并根据管理员的反馈完善博客",
)

executor = autogen.ConversableAgent(
    name="Executor",
    description="执行engineer编写的代码并报告结果。",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

我们上次定义群聊可能会出现发言混乱的情况，我们可以通过添加`allowed_or_disallowed_speaker_transitions`与`speaker_transitions_type`参数来对群聊进行额外的约束，前者可以指定Agent的消息能够/不能够被哪些Agent看到，具体是能够或不能够则可以通过设置`speaker_transitions_type`为`allowed`或`disallowed`来控制。

In [13]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

In [14]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

写一篇关于过去一个月Nvidia股价表现的博客。今天是 2024-04-23。

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

为了写一篇关于过去一个月Nvidia股价表现的博客，我们需要以下信息：

1. **获取Nvidia的股票代码** - 确认Nvidia的股票代码（通常是NVDA）。
2. **获取历史股票数据** - 使用Python代码获取过去一个月的Nvidia股价数据（具体日期范围：2024-03-23至2024-04-23）。
3. **数据处理和可视化** - 对数据进行处理，并创建图表以可视化股价趋势。
4. **市场和行业背景信息** - 获取Nvidia在过去一个月的新闻和行业背景信息，以了解可能影响股价变动的原因。
5. **撰写博客内容** - 将分析结果、数据图表和市场背景信息整合成文，将其撰写成一篇流畅的博客文章。

接下来，我们将按照步骤进行：

### 第一步：确认Nvidia的股票代码
```python
# Nvidia 的股票代码查询
nvidia_ticker = 'NVDA'
print(nvidia_ticker)
```

接下来，请运行上述代码以确认Nvidia的股票代码。如果确认无误，请继续进行第二步。

--------------------------------------------------------------------------------

Next speaker: Engineer

Engineer (to chat_manager):

接下来，我们获取过去一个月的Nvidia股价数据并进行可视化处理。我们将使用Python编写代码，通过Python库如 `yfinance` 获取历史股价数据。

### 第二步：获取历史股票数据

```python
# filename: fetch_nvidia_stock_data.py

import yf

**注意**: 你可能会体验到智能体之间略有不同的交互。工程师Agent可能编写了不正确的代码，执行Agent将报告并发回更正。这个过程可能会经过多轮。